In [2]:
from qiskit import *
from qiskit.visualization import *
from qiskit.transpiler import PassManager
#from qiskit.quantum_info.operators import Operator, Pauli
from qiskit.providers.ibmq import least_busy
from qiskit.aqua.algorithms import ExactEigensolver
from qiskit.aqua.algorithms import QPE
from qiskit.aqua import QuantumInstance
from qiskit.aqua.components.initial_states import Custom
from qiskit.aqua import Operator
from qiskit.aqua.components.iqfts import Standard
import numpy as np
import math

In [3]:
IBMQ.load_accounts()

In [4]:
def matrixHamiltonian(tam, freq): #tam 4, estados 0 a 3
    h = 6.582119624 * 10**(-16)
    E = [h*freq*(i+1/2) for i in range(tam)]
    L = []
    for i in range(tam):
        L.append(list())
        for j in range(tam):
            if i==j: L[i].append(E[i])
            else: L[i].append(0)
    m = np.matrix(L)
    return m


In [23]:
m.item(0)

3.291059812

In [22]:
m = matrixHamiltonian(4, 10**16)
q_bits = 2

for i in range(2**q_bits):
    op = Operator(matrix=m)
    eig_res = ExactEigensolver(op, k=2**q_bits)
    eig_res = eig_res.run()
    eigv = eig_res['eigvecs']

    state = Custom(2, state_vector=eigv[i])

    num_time_slices = 50
    n_ancillae = 11
    iqft = Standard(n_ancillae)

    qpe = QPE(op, state, iqft, num_time_slices, n_ancillae, expansion_mode='suzuki',
              expansion_order=2, shallow_circuit_concat=True)

    backend = BasicAer.get_backend('qasm_simulator')
    quantum_instance = QuantumInstance(backend, shots=100, pass_manager=PassManager())
    result_qpe[i] = qpe.run(quantum_instance)
    print('The ground state energy as computed by QPE is: {}'.format(result_qpe[i]['energy']))

The ground state energy as computed by QPE is: 3.2846319608046883
The ground state energy as computed by QPE is: 9.876393361597657
The ground state energy as computed by QPE is: 16.445657283207026
The ground state energy as computed by QPE is: -23.037418683999995


In [35]:
result_qpe['energy']

3.3296269191718757

In [12]:
[3.3296269191718757, 0.7148437499999998, 0.9296875, -1.0]

In [ ]:
 3.2846319608046883
9.876393361597657
16.445657283207026
-23.037418683999995

In [24]:
m

matrix([[ 3.29105981,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  9.87317944,  0.        ,  0.        ],
        [ 0.        ,  0.        , 16.45529906,  0.        ],
        [ 0.        ,  0.        ,  0.        , 23.03741868]])